<a href="https://colab.research.google.com/github/carlospa6/carlospa6.github.io/blob/master/DBscientist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ⚙️ Install required packages
!pip install pytorch-tabnet scikit-learn pandas matplotlib xgboost tensorflow --quiet

# 📦 Imports
import pandas as pd
import numpy as np
import requests
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import urllib.parse
from google.colab import auth
# auth.authenticate_user()

from IPython.display import Javascript
from google.colab import output
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def fetch_colab_params():
    try:
        icon_id = await output.eval_js('new URLSearchParams(window.location.search).get("icon_id") || ""')
        user_id = await output.eval_js('new URLSearchParams(window.location.search).get("user_id") || ""')
        algorithm = await output.eval_js('new URLSearchParams(window.location.search).get("algorithm") || "tabnet"')

        if not icon_id or not user_id:
            raise ValueError("Missing URL parameters")

        return icon_id, user_id, algorithm
    except Exception as e:
        print("❌ Failed to fetch URL params:", e)
        return None, None, None

icon_id, user_id, ALGORITHM = await fetch_colab_params()

if not icon_id or not user_id:
    raise Exception("❌ Required URL parameters missing. Please pass icon_id and user_id in the Colab URL.")

print(f"📌 icon_id: {icon_id}, user_id: {user_id}, algorithm: {ALGORITHM}")

def get_url_param(name):
    return output.eval_js(f"""
    new Promise((resolve) => {{
        const urlParams = new URLSearchParams(window.location.search);
        resolve(urlParams.get("{name}"));
    }})
    """)



# 🔁 Optional: hyperparameter overrides
HYPERPARAMS = {
    "epochs": 50,
    "learning_rate": 0.01,
    "batch_size": 32,
    "hidden_units": 64  # used in MLP/DCN
}

# 📥 Load JSON data
response = requests.get(PROCESSED_JSON_URL)
if response.status_code != 200:
    raise Exception(f"❌ Failed to load JSON: {response.status_code}")
data = response.json()
df = pd.DataFrame(data)
print(f"✅ Loaded dataset: {df.shape}")

# 🎯 Select target column (must match your processing stage)
TARGET_COLUMN = "target"

# 📊 Prepare Data
X = pd.get_dummies(df.drop(columns=[TARGET_COLUMN]), drop_first=True)
y = df[TARGET_COLUMN].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

# 🚀 Train model
if ALGORITHM == 'tabnet':
    clf = TabNetClassifier(
        n_d=8, n_a=8,
        n_steps=3,
        gamma=1.3,
        lambda_sparse=1e-3,
        optimizer_params=dict(lr=HYPERPARAMS['learning_rate']),
        verbose=1
    )
    clf.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        max_epochs=HYPERPARAMS["epochs"],
        batch_size=HYPERPARAMS["batch_size"],
        virtual_batch_size=16
    )
    y_pred = clf.predict(X_test)

elif ALGORITHM == 'mlp':
    clf = MLPClassifier(
        hidden_layer_sizes=(HYPERPARAMS['hidden_units'],),
        max_iter=HYPERPARAMS["epochs"],
        learning_rate_init=HYPERPARAMS["learning_rate"],
        batch_size=HYPERPARAMS["batch_size"],
        verbose=True
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

elif ALGORITHM == 'dcn':
    input_dim = X_train.shape[1]
    inputs = keras.Input(shape=(input_dim,))

    # Cross Layer
    cross = layers.Dense(1, activation=None)(inputs)

    # Deep Layer
    deep = layers.Dense(HYPERPARAMS['hidden_units'], activation='relu')(inputs)
    deep = layers.Dense(HYPERPARAMS['hidden_units'], activation='relu')(deep)

    concat = layers.concatenate([cross, deep])
    outputs = layers.Dense(len(np.unique(y)), activation='softmax')(concat)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=HYPERPARAMS["learning_rate"]),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=HYPERPARAMS["epochs"], batch_size=HYPERPARAMS["batch_size"], validation_split=0.2)
    y_pred = np.argmax(model.predict(X_test), axis=1)

else:
    raise ValueError("❌ Invalid algorithm selected. Choose 'tabnet', 'mlp', or 'dcn'.")

# 📈 Evaluate
print("\n📋 Classification Report:\n")
print(classification_report(y_test, y_pred))

# 💾 Optional: Save model
if ALGORITHM == 'tabnet':
    clf.save_model("tabnet_model.zip")
elif ALGORITHM == 'mlp':
    import joblib
    joblib.dump(clf, "mlp_model.pkl")
elif ALGORITHM == 'dcn':
    model.save("dcn_model.h5")

print("✅ Model saved.")

TypeError: object NoneType can't be used in 'await' expression